In [1]:
import os
import time
import bz2
import codecs
import pickle as pkl

import numpy as np
from scipy import stats

from models import *

In [2]:
dep_based = load_model("deps.words")
bow2_based = load_model("bow2.words")
bow5_based = load_model("bow5.words")

In [5]:
all_models = [['Dependency', dep_based], 
              ['BoW with k = 2', bow2_based], 
              ['BoW with k = 5', bow5_based]]

In [11]:
# Read Simlex999.txt 
def read_sim_file(filename):
    pairs_list = []
    pairs_dict = {}
    norm_dict = {}

    with open(filename, "r") as f:
        for _, line in enumerate(f):
            line = line.strip().split()
            pairs_list.append((line[0], line[1], line[3]))
            pairs = pairs_list[1:]
    for word1, word2, score in pairs:
        if word1 not in pairs_dict.keys():
            pairs_dict[word1] = []
            norm_dict[word1] = []
        pairs_dict[word1].append((word2,float(score)))

#     for key, value in pairs_dict.items():
#         normalized_sum = 0.0
#         for _, score in value:
#             normalized_sum += score
#         for w, score in value:
#             score = round(score/normalized_sum, 3)
#             norm_dict[key].append((w, score))

#     Sorting by scores
    for word1, list_value in pairs_dict.items():
        list_value.sort(key=lambda _: -_[1])
    
    return pairs_dict

In [24]:
#Read MEN-Dataset-Natural-Full-Form
def read_men_file(filename):
    pairs_list = []
    pairs_dict = {}
    norm_dict = {}
    
    with open(filename, "r") as f:
        for _, line in enumerate(f):
            line = line.strip().split()
            pairs_list.append((line[0], line[1], line[2]))
    for word1, word2, score in pairs_list:
        if word1 not in pairs_dict.keys():
            pairs_dict[word1] = []
            norm_dict[word1] = []
        pairs_dict[word1].append((word2, float(score)/5.0))
    
# #     #normalizing scores in [0,1]
#     for key, value in pairs_dict.items():
#         normalized_sum = 0.0
#         for _, score in value:
#             normalized_sum += score
#         for w, score in value:
#             score = round(score/normalized_sum, 3)
#             norm_dict[key].append((w, score))
        
    for key, value in pairs_dict.items():
        value.sort(key=lambda _: -_[1])
        
    return pairs_dict

In [25]:
men_natural = read_men_file("data/MEN/MEN_dataset_natural_form_full")

In [26]:
men_natural["happy"]

[('smile', 8.0),
 ('love', 7.8),
 ('kids', 5.8),
 ('sexy', 5.6),
 ('tears', 5.4),
 ('kiss', 5.2),
 ('mom', 5.2),
 ('time', 4.8),
 ('home', 4.6),
 ('morning', 4.4),
 ('idea', 4.4),
 ('sleep', 4.4),
 ('stand', 3.4),
 ('night', 3.4),
 ('toes', 3.2),
 ('posted', 2.8),
 ('walk', 2.8),
 ('sitting', 2.2),
 ('licking', 2.2),
 ('stop', 1.0),
 ('rusty', 0.6)]

In [27]:
simlex_999 = read_sim_file("data/SimLex-999/SimLex-999.txt")

In [28]:
simlex_999["happy"]

[('cheerful', 9.55),
 ('glad', 9.17),
 ('young', 2.0),
 ('angry', 1.28),
 ('mad', 0.95)]

In [35]:
eval_models = [["MEN (Natural) Full Form", men_natural],
              ["SimLex-999", simlex_999]]

In [37]:
# Evaluation
# For each common word from Simlex vs. models
#     Find length of Simlex data
#     Read similar length data in models
#     Store scores in (a, b) - for spearman (two 1D arrays of the scores)

for item in eval_models:
    start_time = time.time()
    for comp_item in all_models:
        a = np.zeros((len(item[1]),1))
        b = np.zeros((len(item[1]),1))
        i = 0
        for key, value in item[1].items():
            for model_word in comp_item[1].word_index.keys():
                if (key == model_word):
                    eval_length = len(value)
                    model_all_values = comp_item[1].most_similar(key, score = True, n = eval_length)
                    
                    for _, score in value:
                        a[i] = score
                    for _, model_score in model_all_values:
                        b[i] = model_score
            if(i%100 == 0):
                print(i)
            i=i+1
            rho, _ = stats.spearmanr(a, b, axis = None)
            val, _ = stats.pearsonr(a, b)
        print(item[0], ' vs. ', comp_item[0], ' ~ Spearman Correlation: ', rho)
        print(item[0], ' vs. ', comp_item[0], ' ~ Pearson Correlation: ', val)
    print("Comparing against ", item[0], " for all models took ", time.time() - start_time, " seconds.\n")

0
100
200
300
400
500
600
MEN (Natural) Full Form  vs.  Dependency  ~ Spearman Correlation:  0.012574605774151478
MEN (Natural) Full Form  vs.  Dependency  ~ Pearson Correlation:  [0.08847134]
0
100
200
300
400
500
600
MEN (Natural) Full Form  vs.  BoW with k = 2  ~ Spearman Correlation:  0.11644121914190454
MEN (Natural) Full Form  vs.  BoW with k = 2  ~ Pearson Correlation:  [0.17544529]
0
100
200
300
400
500
600
MEN (Natural) Full Form  vs.  BoW with k = 5  ~ Spearman Correlation:  0.11166579992975292
MEN (Natural) Full Form  vs.  BoW with k = 5  ~ Pearson Correlation:  [0.15944745]
Comparing against  MEN (Natural) Full Form  for all models took  4249.335703611374  seconds.

0


KeyboardInterrupt: 